In [3]:
!git clone https://github.com/WMARBAR/Mundial_Inferences.git

fatal: destination path 'Mundial_Inferences' already exists and is not an empty directory.


In [4]:
import sys
sys.path.append('/content/Mundial_Inferences')  # Asegúrate que esta ruta es correcta

# 3. PRUEBAS DE HIPOTESIS

# LIBS

In [5]:
# ==========================================
# Análisis completo de jugadores mundialistas
# Basado en confederación y condición de campeón
# ==========================================


from exps.loaders import Loaders
from exps.mundial_exps_extended import Experimentos
from exps.dists import AnalisisEstadistico
from exps.dists import Muestreos
import pandas as pd
import scipy.stats as stats
ld=Loaders()
exper=Experimentos()
muestreos= Muestreos()


# Cargar archivo Excel
path = '/content/Mundial_Inferences/datain/Proyecto_mundial.xlsx'
df_mundial= ld.cargar_excel(path,'Sheet1')

# Inicializar clases
dist = AnalisisEstadistico(df_mundial)
exper = Experimentos()

n=2500

## HIPOTESIS 1: LOS CAMPEONES TIENEN MÁS DEFENSAS CON CAPACIDAD DE HACER GOL

### Se genera de df_mundial la base df_equipos que contiene el pais, si fue campeon y los goles por defensas

In [6]:
# Filtrar jugadores que son defensas y anotaron goles
df_defensas_anotadores = df_mundial[
    (df_mundial['categ_posicion'].str.contains("Defensa", case=False, na=False)) &
    (df_mundial['Goles Marcados(mundial)'] == 1)
]
df_defensas_anotadores_por_equipo = df_defensas_anotadores.groupby('Seleccion').size().reset_index(name='n_defensas_anotadores')
# Agrupamos para saber si una selección fue campeona
df_campeones = df_mundial.groupby('Seleccion')['dummy_campeon'].max().reset_index()

# Unimos con el conteo de defensas anotadores
df_equipos = pd.merge(df_campeones, df_defensas_anotadores_por_equipo, on='Seleccion', how='left')

# Los que no tienen defensas anotadores quedan con NaN, se reemplazan por 0
df_equipos['n_defensas_anotadores'] = df_equipos['n_defensas_anotadores'].fillna(0)




In [7]:
from scipy.stats import ttest_ind

# Separar los grupos
grupo_campeon = df_equipos[df_equipos['dummy_campeon'] == 1]['n_defensas_anotadores']
grupo_no_campeon = df_equipos[df_equipos['dummy_campeon'] == 0]['n_defensas_anotadores']

# Prueba t de dos muestras independientes, asumiendo varianzas distintas
t_stat, p_valor_dos_colas = ttest_ind(grupo_campeon, grupo_no_campeon, equal_var=False)

# Como queremos una cola (mayor), dividimos p-valor por 2 y revisamos el signo del estadístico
p_valor_una_cola = p_valor_dos_colas / 2

print("Media campeones:", grupo_campeon.mean())
print("Media no campeones:", grupo_no_campeon.mean())
print("Estadístico t:", t_stat)
print("p-valor (una cola):", p_valor_una_cola)

if p_valor_una_cola < 0.05 and t_stat > 0:
    print("✅ Se rechaza H₀: los campeones tienen significativamente más defensas anotadores.")
else:
    print("❌ No se rechaza H₀: no se prueba que los campeones tengan más defensas anotadores.")


Media campeones: 4.75
Media no campeones: 0.5
Estadístico t: 3.5989923540442947
p-valor (una cola): 0.004264662087944536
✅ Se rechaza H₀: los campeones tienen significativamente más defensas anotadores.


## HIPOTESIS 2: SUPERAR EL GRUPO DE LA MUERTE AUMENTA EL CHANCE DE LLEGAR A LA FINAL

In [8]:
from statsmodels.stats.proportion import proportions_ztest

# Filtrar valores
grupo1 = df_mundial[df_mundial["Grupo_De_La_Muerte"] == 1]["dummy_finalistas"]
grupo2 = df_mundial[df_mundial["Grupo_De_La_Muerte"] == 0]["dummy_finalistas"]

# Número de éxitos (finalistas) y tamaño de muestra
successes = [grupo1.sum(), grupo2.sum()]
nobs = [len(grupo1), len(grupo2)]

# Prueba de proporciones (una cola: grupo 1 > grupo 2)
z_stat, p_val = proportions_ztest(successes, nobs, alternative='larger')

# Resultados
print(f"Proporción grupo de la muerte: {grupo1.mean():.4f}")
print(f"Proporción grupo normal: {grupo2.mean():.4f}")
print(f"Estadístico z: {z_stat:.4f}")
print(f"p-valor (una cola): {p_val:.4f}")

if p_val < 0.05:
    print("✅ Se rechaza H₀: estar en el grupo de la muerte se asocia con mayor probabilidad de llegar a la final.")
else:
    print("❌ No se rechaza H₀: no hay evidencia suficiente para afirmar que influye.")


Proporción grupo de la muerte: 0.1904
Proporción grupo normal: 0.0764
Estadístico z: 13.8300
p-valor (una cola): 0.0000
✅ Se rechaza H₀: estar en el grupo de la muerte se asocia con mayor probabilidad de llegar a la final.


## HIPOTESIS 3: LA EDAD INFLUYE EN LOS CAMPEONES

In [9]:
media_campeones = df_mundial[df_mundial['dummy_campeon'] == 1]['edad_Player'].mean()
media_no_campeones = df_mundial[df_mundial['dummy_campeon'] == 0]['edad_Player'].mean()
print(f"Edad promedio campeones: {media_campeones:.2f}")
print(f"Edad promedio NO campeones: {media_no_campeones:.2f}")


Edad promedio campeones: 26.84
Edad promedio NO campeones: 26.51


In [10]:
from scipy import stats
import numpy as np

grupo1 = df_mundial[df_mundial['dummy_campeon'] == 1]['edad_Player'].dropna()
grupo2 = df_mundial[df_mundial['dummy_campeon'] == 0]['edad_Player'].dropna()

def intervalo_confianza(data, confianza=0.95):
    n = len(data)
    media = np.mean(data)
    sem = stats.sem(data)
    h = sem * stats.t.ppf((1 + confianza) / 2, n - 1)
    return (media - h, media + h)

ic_campeones = intervalo_confianza(grupo1)
ic_no_campeones = intervalo_confianza(grupo2)

print(f"IC edad campeones: {ic_campeones}")
print(f"IC edad NO campeones: {ic_no_campeones}")


IC edad campeones: (np.float64(26.50975143092788), np.float64(27.17803137450198))
IC edad NO campeones: (np.float64(26.436776266577162), np.float64(26.582198208236484))


In [11]:
t_stat, p_value = stats.ttest_ind(grupo1, grupo2, equal_var=False)
print(f"Prueba t: t = {t_stat:.4f}, p = {p_value:.4f}")
if p_value < 0.05:
    print("❌ Se rechaza H₀: hay diferencia significativa en la edad.")
else:
    print("✅ No se rechaza H₀: no hay evidencia suficiente de diferencia en la edad.")


Prueba t: t = 1.9217, p = 0.0552
✅ No se rechaza H₀: no hay evidencia suficiente de diferencia en la edad.


#  HIPOTESIS 4: comparar si existen diferencias significativas en las edades promedio de los jugadores entre las distintas confederaciones (ANOVA)

In [12]:
from scipy.stats import f_oneway

grupos = [df_mundial[df_mundial["confederacion"] == conf]["edad_enMundial"].dropna()
          for conf in df_mundial["confederacion"].unique()]

f_stat, p_valor = f_oneway(*grupos)

print("Estadístico F:", f_stat)
print("p-valor:", p_valor)

if p_valor < 0.05:
    print("✅ Se rechaza H₀: hay diferencias significativas entre confederaciones.")
else:
    print("❌ No se rechaza H₀: no se encontraron diferencias significativas.")


Estadístico F: 43.08181015083578
p-valor: 3.4767969602009204e-52
✅ Se rechaza H₀: hay diferencias significativas entre confederaciones.


#  HIPOTESIS 5 : comparar si existen diferencias significativas en los años de experiencia en el mundial de los jugadores entre las distintas confederaciones (ANOVA)

In [13]:
from scipy.stats import f_oneway

grupos = [df_mundial[df_mundial["confederacion"] == conf]["years_expMundial"].dropna()
          for conf in df_mundial["confederacion"].unique()]

f_stat, p_valor = f_oneway(*grupos)

print("Estadístico F:", f_stat)
print("p-valor:", p_valor)

if p_valor < 0.05:
    print("✅ Se rechaza H₀: hay diferencias significativas entre confederaciones.")
else:
    print("❌ No se rechaza H₀: no se encontraron diferencias significativas.")


Estadístico F: 0.6691730639248618
p-valor: 0.6746416457528465
❌ No se rechaza H₀: no se encontraron diferencias significativas.
